# <center> PJM Generation by Fuel: Data Processing </center> 

In [1]:
import pandas as pd
import os
import datetime

## Read in and format data 

In [2]:
gen_by_fuel_df = pd.read_csv(os.path.join(os.pardir, 'raw_data', 'gen_by_fuel.csv'), index_col=0, parse_dates=[0])
gen_by_fuel_df.drop('datetime_beginning_ept', axis=1, inplace=True)
gen_by_fuel_df.index.name = 'DATE_UTC'

In [3]:
# actually need to put in UTC-5
gen_by_fuel_df.index = gen_by_fuel_df.index + datetime.timedelta(hours=-5)

In [4]:
gen_by_fuel_df = gen_by_fuel_df.sort_index()
gen_by_fuel_df.columns = ['FUEL_TYPE', 'Generation (MW)', 'PERCENT_TOTAL', 'IS_RENEWABLE']

In [5]:
gen_by_fuel_df.head()

,FUEL_TYPE,Generation (MW),PERCENT_TOTAL,IS_RENEWABLE
DATE_UTC,,,,
2015-01-01 00:00:00,Other,96035.3,1.0,False
2015-01-01 01:00:00,Other,93448.1,1.0,False
2015-01-01 02:00:00,Other,91809.4,1.0,False
2015-01-01 03:00:00,Other,90742.9,1.0,False
2015-01-01 04:00:00,Other,90409.2,1.0,False


In [6]:
# Take a look at averages
gen_by_fuel_df.loc['2016-01-01':].groupby('FUEL_TYPE').mean()

,Generation (MW),PERCENT_TOTAL,IS_RENEWABLE
FUEL_TYPE,,,
Coal,29595.754934,0.317774,False
Flywheel,0.000000,0.000000,False
Gas,23056.781469,0.248543,False
Hydro,1544.946811,0.015982,True
Multiple Fuels,816.320523,0.008319,False
Nuclear,32329.479566,0.360474,False
Oil,304.540705,0.001273,False
Other,1040.272778,0.010600,False
Other Renewables,623.031903,0.004876,True


In [7]:
gen_by_fuel_df = pd.DataFrame(gen_by_fuel_df.loc['2016-01-01':])

## Verify data 

In [8]:
# A bunch of data points have zeros
totals = gen_by_fuel_df.groupby(gen_by_fuel_df.index).sum()
totals[(totals['PERCENT_TOTAL'] > 1.02) | (totals['PERCENT_TOTAL'] < 0.97)].head()

,Generation (MW),PERCENT_TOTAL,IS_RENEWABLE
DATE_UTC,,,
2016-02-03 00:00:00,0.0,0.0,0.0
2016-02-03 01:00:00,0.0,0.0,0.0
2016-02-03 02:00:00,0.0,0.0,0.0
2016-02-03 03:00:00,0.0,0.0,0.0
2016-02-03 04:00:00,0.0,0.0,0.0


In [9]:
# Drop zeros
gen_by_fuel_df = gen_by_fuel_df.drop(totals[totals['Generation (MW)'] == 0].index)

In [10]:
# Data points aren't exactly 100% percent total (though all close), so we'll scale them
totals = gen_by_fuel_df.groupby(gen_by_fuel_df.index).sum()
totals[(totals['PERCENT_TOTAL'] > 1.005) | (totals['PERCENT_TOTAL'] < 0.995)].head()

,Generation (MW),PERCENT_TOTAL,IS_RENEWABLE
DATE_UTC,,,
2016-01-01 00:00:00,77359.0,1.01,4.0
2016-01-01 03:00:00,74021.7,1.01,4.0
2016-01-01 04:00:00,73874.1,1.01,4.0
2016-01-01 05:00:00,75140.7,1.01,4.0
2016-01-01 06:00:00,74510.9,1.01,4.0


## Scale percentages (which are all close to but not all quite 100%) 

In [11]:
gen_scaled = \
    gen_by_fuel_df.groupby(gen_by_fuel_df.index).apply(lambda x: x['Generation (MW)']/(x['PERCENT_TOTAL'].sum()))
pct_scaled = \
    gen_by_fuel_df.groupby(gen_by_fuel_df.index).apply(lambda x: x['PERCENT_TOTAL']/(x['PERCENT_TOTAL'].sum()))

In [12]:
gen_scaled.index = gen_by_fuel_df.index
pct_scaled.index = gen_by_fuel_df.index

In [13]:
gen_by_fuel_df['Generation (MW)'] = gen_scaled
gen_by_fuel_df['PERCENT_TOTAL']   = pct_scaled

## Verify that percentages are correct 

In [14]:
# Our calculation of percentage
percent_calc = gen_by_fuel_df.groupby(gen_by_fuel_df.index)['PERCENT_TOTAL'].apply(lambda x: x/x.sum())

In [15]:
# PJM's calculation of percentage
percent_rep = gen_by_fuel_df['PERCENT_TOTAL']

In [16]:
# They look close to each other, so we'll keep PJM's reported percentages
diffs = percent_calc - percent_rep
print(diffs.max())
print(diffs.min())
print(diffs.mean())
print(diffs.std())

1.6653345369377348e-16
-1.1102230246251565e-16
1.4403060946217052e-18
1.399623586247929e-17


## Reindex 

In [17]:
gen_by_fuel_df = gen_by_fuel_df.set_index([gen_by_fuel_df.index, 'FUEL_TYPE'])

In [18]:
# For all hours and dates
date_range = pd.date_range(start='2016-01-01 05:00', end='2018-01-01 5:00', freq='H')
fuel_types = gen_by_fuel_df.index.get_level_values('FUEL_TYPE').unique()
multi_idx = pd.MultiIndex.from_product([date_range, fuel_types], names=['DATE_UTC', 'FUEL_TYPE'])

In [19]:
gen_by_fuel_df = gen_by_fuel_df.reindex(multi_idx).sort_index()